# Analyse des flux RSS - InsightDetector

Ce notebook effectue une analyse complète des articles collectés depuis les flux RSS configurés.

## Sommaire

1. [Configuration et imports](#1-configuration-et-imports)
2. [Collecte des données](#2-collecte-des-données) 
3. [Traitement et stockage](#3-traitement-et-stockage)
4. [Analyses statistiques de base](#4-analyses-statistiques-de-base)
5. [Analyses temporelles avancées](#5-analyses-temporelles-avancées)
6. [Analyse des mots-clés et contenus](#6-analyse-des-mots-clés-et-contenus)
7. [Visualisations interactives](#7-visualisations-interactives)
8. [Métriques de qualité](#8-métriques-de-qualité)
9. [Conclusions et recommandations](#9-conclusions-et-recommandations)

---

## 1. Configuration et imports



In [20]:
# Auto-reload des modules modifiés (pratique pour développement)
%load_ext autoreload
%autoreload 2

# Imports standards
import sys
from pathlib import Path

# Définition du répertoire racine du projet 
project_root = Path().resolve().parents[1] if "notebooks" in str(Path().resolve()) else Path().resolve()

# Définition des chemins utilisés
src_dir = project_root / "src"
data_dir = project_root / "data"
outputs_dir = project_root / "outputs"
notebooks_dir = project_root / "notebooks"

# Création des dossiers s'ils n'existent pas
for d in [data_dir, outputs_dir]:
    d.mkdir(parents=True, exist_ok=True)

# 
src_path = str(src_dir.resolve())
if src_path not in sys.path:
    sys.path.insert(0, src_path)




The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
# Imports principaux
from data.rss_collector import RSSCollector
from data.article_parser import ArticleParser
from data.database_manager import DatabaseManager
from config.database import SessionLocal

import pandas as pd
import numpy as np
import time
import logging
from datetime import datetime, timedelta
import warnings
from collections import Counter
import re

# Visualisation
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots


In [22]:

import os
from dotenv import load_dotenv
from sqlalchemy import create_engine

# Charger .env
load_dotenv(dotenv_path="../../.env")  # adapte le chemin si besoin

# Récupérer la vraie DATABASE_URL
db_url = os.getenv("DATABASE_URL")
if db_url is None:
    raise ValueError("DATABASE_URL manquant dans le fichier .env")

# Création moteur SQLAlchemy
connect_args = {"check_same_thread": False} if db_url.startswith("sqlite") else {}
engine = create_engine(db_url, connect_args=connect_args)

# Test connexion
with engine.connect() as conn:
    print(f"Connexion réussie à la base : {db_url}")


Connexion réussie à la base : sqlite:///data/insight.db


In [23]:
# Pour accéder au dossier src/ depuis le notebook
import sys
sys.path.append(".")

# Importation des composants ORM
from config.database import engine
from data.models import Base

# Création des tables dans la base PostgreSQL
Base.metadata.create_all(bind=engine)



In [24]:
from sqlalchemy import text

with engine.connect() as conn:
    result = conn.execute(
        text("SELECT name FROM sqlite_master WHERE type='table';")  # pour SQLite
    )
    print("Tables dans la base de données :")
    for row in result:
        print("  →", row[0])
        


Tables dans la base de données :
  → articles


In [25]:
# Installation et imports avec gestion d'erreurs
import sys
import subprocess
import importlib

# Function pour installer les packages manquants
def install_package(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# Liste des packages requis
required_packages = [
    "lxml_html_clean",
    "pandas", 
    "matplotlib", 
    "seaborn", 
    "plotly", 
    "feedparser",
    "beautifulsoup4"
]

# Installer les packages manquants
for package in required_packages:
    try:
        importlib.import_module(package)
    except ImportError:
        print(f"Installation de {package}...")
        try:
            install_package(package)
        except:
            print(f"Impossible d'installer {package}")

# Imports principaux avec gestion d'erreur
try:
    import sys
    sys.path.append('C:/Users/beedi.goua_square-ma/Desktop/Gheb/projet perso/InsightDetector/insight-detector/src')
    
    from data.rss_collector import RSSCollector
    from data.article_parser import ArticleParser
    from data.database_manager import DatabaseManager
    from config.database import SessionLocal
except ImportError as e:
    print(f"Erreur d'import modules projet: {e}")
    print(" Vérifiez que vous êtes dans le bon répertoire et que les modules existent")


Installation de beautifulsoup4...


In [26]:

import pandas as pd
import numpy as np
import time
import logging
from datetime import datetime, timedelta
import warnings
from collections import Counter
import re

# Visualisation (avec gestion d'erreurs)
try:
    import matplotlib.pyplot as plt
    import seaborn as sns
    plt.style.use('seaborn-v0_8')
except:
    plt.style.use('default')
    
try:
    import plotly.express as px
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    plotly_available = True
except ImportError:
    print("Plotly non disponible - visualisations interactives désactivées")
    plotly_available = False

# Configuration
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

# Configuration logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Flux RSS utilisés
RSS_SOURCES = [
    "https://www.lemonde.fr/rss/une.xml",
    "https://www.france24.com/fr/rss", 
    "https://www.rfi.fr/fr/rss",
    "https://www.lexpress.fr/rss/actualite.xml",
    "https://www.courrierinternational.com/feed/all/rss.xml",
    "https://www.nouvelobs.com/rss.xml",
    "https://www.lesechos.fr/rss/rss_une.xml",
    "https://www.euronews.com/rss?level=theme&name=news",
    "https://www.huffingtonpost.fr/feeds/index.xml",
    "https://rss.nytimes.com/services/xml/rss/nyt/World.xml"
]

print("Configuration et imports terminés")
print(f" Plotly disponible: {'Oui' if plotly_available else 'Non'}")
print(f"  Sources RSS configurées: {len(RSS_SOURCES)}")

Configuration et imports terminés
 Plotly disponible: Oui
  Sources RSS configurées: 10


In [27]:
# Initialisation des composants avec gestion d'erreurs
session = None
collector = None
parser = None
db = None

try:
    collector = RSSCollector(RSS_SOURCES)
    parser = ArticleParser()
    session = SessionLocal()
    db = DatabaseManager(session)
    logger.info("Composants initialisés avec succès")
    
    # Variables pour tracking des métriques
    metrics = {
        'sources_success': {},
        'sources_errors': {},
        'total_fetched': 0,
        'total_processed': 0,
        'total_inserted': 0,
        'total_duplicates': 0,
        'processing_errors': []
    }
    
except Exception as e:
    logger.error(f"Erreur lors de l'initialisation: {e}")
    if session:
        session.close()
    raise

2025-07-23 10:15:06,070 - INFO - Composants initialisés avec succès


## 2. Collecte des données

Initialisation des composants et collecte des articles depuis les flux RSS configurés.

In [28]:
# Collecte robuste des articles avec métriques détaillées
articles = []
start_time = time.time()

try:
    logger.info("Début de la collecte des flux RSS...")
    articles = collector.fetch_feeds()
    metrics['total_fetched'] = len(articles)
    
    # Analyse des sources par succès/échec
    for article in articles:
        source = article.get('source', 'Unknown')
        metrics['sources_success'][source] = metrics['sources_success'].get(source, 0) + 1
    
    collection_time = time.time() - start_time
    
    logger.info(f"{len(articles)} articles collectés en {collection_time:.2f}s")
    print(f"Résumé de la collecte:")
    print(f"   • Total articles: {len(articles)}")
    print(f"   • Temps de collecte: {collection_time:.2f}s")
    print(f"   • Articles par seconde: {len(articles)/collection_time:.1f}")
    
    # Affichage par source
    if articles:
        print("\nArticles par source:")
        for source, count in sorted(metrics['sources_success'].items(), key=lambda x: x[1], reverse=True):
            print(f"   • {source}: {count} articles")
    
except Exception as e:
    logger.error(f"Erreur lors de la collecte: {e}")
    metrics['processing_errors'].append(f"Collecte: {str(e)}")
    print(f"Échec de la collecte: {e}")

2025-07-23 10:15:06,171 - INFO - Début de la collecte des flux RSS...
2025-07-23 10:15:06,171 - INFO - Lecture du flux RSS : https://www.lemonde.fr/rss/une.xml
2025-07-23 10:15:06,471 - INFO - Lecture du flux RSS : https://www.france24.com/fr/rss
2025-07-23 10:15:06,653 - INFO - Lecture du flux RSS : https://www.rfi.fr/fr/rss
2025-07-23 10:15:06,823 - INFO - Lecture du flux RSS : https://www.lexpress.fr/rss/actualite.xml
2025-07-23 10:15:07,679 - INFO - Lecture du flux RSS : https://www.courrierinternational.com/feed/all/rss.xml
2025-07-23 10:15:07,912 - INFO - Lecture du flux RSS : https://www.nouvelobs.com/rss.xml
2025-07-23 10:15:08,245 - INFO - Lecture du flux RSS : https://www.lesechos.fr/rss/rss_une.xml
2025-07-23 10:15:08,411 - INFO - Lecture du flux RSS : https://www.euronews.com/rss?level=theme&name=news
2025-07-23 10:15:08,565 - INFO - Lecture du flux RSS : https://www.huffingtonpost.fr/feeds/index.xml
2025-07-23 10:15:08,877 - INFO - Lecture du flux RSS : https://rss.nytimes

Résumé de la collecte:
   • Total articles: 510
   • Temps de collecte: 2.95s
   • Articles par seconde: 173.1

Articles par source:
   • https://www.nouvelobs.com/rss.xml: 200 articles
   • https://www.lexpress.fr/rss/actualite.xml: 100 articles
   • https://rss.nytimes.com/services/xml/rss/nyt/World.xml: 54 articles
   • https://www.euronews.com/rss?level=theme&name=news: 50 articles
   • https://www.france24.com/fr/rss: 24 articles
   • https://www.rfi.fr/fr/rss: 24 articles
   • https://www.courrierinternational.com/feed/all/rss.xml: 20 articles
   • https://www.huffingtonpost.fr/feeds/index.xml: 20 articles
   • https://www.lemonde.fr/rss/une.xml: 18 articles


In [29]:
# Traitement optimisé des articles avec métriques détaillées
processed = []
processing_start = time.time()

if articles:
    logger.info(f"Début du traitement de {len(articles)} articles...")
    
    for i, article in enumerate(articles, 1):
        try:
            # Extraction du texte avec timeout
            start_extract = time.time()
            article["text"] = parser.extract_text(article["url"])
            extract_time = time.time() - start_extract
            
            if not article["text"] or len(article["text"]) < 100:
                logger.debug(f"Article {i}: texte trop court ou vide ({article.get('url', 'URL inconnue')})")
                continue
            
            # Tentative d'insertion en base
            try:
                db.create_article(article)
                metrics['total_inserted'] += 1
                processed.append(article)
                logger.debug(f"Article {i}: inséré avec succès (extraction: {extract_time:.2f}s)")
                
            except Exception as db_error:
                if "UNIQUE constraint failed" in str(db_error) or "duplicate key" in str(db_error):
                    metrics['total_duplicates'] += 1
                    logger.debug(f"Article {i}: doublons détecté")
                else:
                    error_msg = f"Insertion article {i}: {str(db_error)}"
                    metrics['processing_errors'].append(error_msg)
                    logger.warning(error_msg)
            
            metrics['total_processed'] += 1
            
            # Pause adaptative pour éviter la surcharge
            time.sleep(0.1 if i % 10 != 0 else 0.3)
            
            # Progress indicator
            if i % 10 == 0 or i == len(articles):
                progress = (i / len(articles)) * 100
                print(f"Progression: {i}/{len(articles)} ({progress:.1f}%) - "
                      f"Insérés: {metrics['total_inserted']}, "
                      f"Doublons: {metrics['total_duplicates']}")
                
        except Exception as e:
            error_msg = f"Erreur traitement article {i}: {str(e)}"
            metrics['processing_errors'].append(error_msg)
            logger.error(error_msg)
            continue

processing_time = time.time() - processing_start

# Résumé final
print(f"\nRésumé du traitement:")
print(f"   • Articles traités: {metrics['total_processed']}/{len(articles)}")
print(f"   • Articles insérés: {metrics['total_inserted']}")
print(f"   • Doublons détectés: {metrics['total_duplicates']}")
print(f"   • Erreurs: {len(metrics['processing_errors'])}")
print(f"   • Temps total: {processing_time:.2f}s")
print(f"   • Taux de succès: {(metrics['total_inserted']/len(articles)*100):.1f}%")

if metrics['processing_errors']:
    print(f"\nDernières erreurs:")
    for error in metrics['processing_errors'][-3:]:  # Affiche les 3 dernières erreurs
        print(f"   • {error}")
        
logger.info(f"Traitement terminé: {metrics['total_inserted']} articles insérés")

2025-07-23 10:15:09,245 - INFO - Début du traitement de 510 articles...
2025-07-23 10:15:09,245 - INFO - Téléchargement de l'article : https://www.lemonde.fr/international/article/2025/07/23/en-cisjordanie-l-expulsion-silencieuse-des-bedouins-sous-la-pression-des-colons-israeliens_6623127_3210.html
2025-07-23 10:15:09,792 - INFO - Téléchargement de l'article : https://www.lemonde.fr/sport/article/2025/07/23/l-entraineur-de-ski-joel-chenal-vise-par-une-plainte-pour-agression-sexuelle-sur-mineure-et-de-nouvelles-accusations_6623140_3242.html
2025-07-23 10:15:10,191 - INFO - Téléchargement de l'article : https://www.lemonde.fr/international/live/2025/07/23/en-direct-guerre-en-ukraine-volodymyr-zelensky-assure-que-les-instances-anticorruption-ukrainiennes-vont-continuer-a-faire-leur-travail-apres-l-adoption-d-une-loi-revenant-sur-leur-independance_6622319_3210.html
2025-07-23 10:15:10,824 - INFO - Téléchargement de l'article : https://www.lemonde.fr/planete/article/2025/07/23/la-loi-duplom

Progression: 10/510 (2.0%) - Insérés: 10, Doublons: 0


2025-07-23 10:15:14,026 - INFO - Téléchargement de l'article : https://www.lemonde.fr/international/article/2025/07/23/en-espagne-pedro-sanchez-echoue-a-faire-adopter-une-loi-contre-les-megapannes-d-electricite_6623132_3210.html
2025-07-23 10:15:14,393 - INFO - Téléchargement de l'article : https://www.lemonde.fr/international/article/2025/07/23/droits-de-douane-donald-trump-annonce-un-accord-avec-le-japon_6623054_3210.html
2025-07-23 10:15:14,785 - INFO - Téléchargement de l'article : https://www.lemonde.fr/international/article/2025/07/23/aux-etats-unis-l-universite-columbia-sanctionne-a-des-etudiants-ayant-participe-a-des-actions-propalestiniennes_6623121_3210.html
2025-07-23 10:15:15,241 - INFO - Téléchargement de l'article : https://www.lemonde.fr/sport/article/2025/07/23/venus-williams-s-offre-a-45-ans-une-victoire-au-tournoi-wta-500-de-washington_6623122_3242.html
2025-07-23 10:15:15,638 - INFO - Téléchargement de l'article : https://www.lemonde.fr/disparitions/article/2025/07/2

Progression: 20/510 (3.9%) - Insérés: 20, Doublons: 0


2025-07-23 10:15:17,894 - INFO - Téléchargement de l'article : https://www.france24.com/fr/vid%C3%A9o/20250723-le-roi-du-heavy-metal-ozzy-osbourne-s-%C3%A9teint-%C3%A0-76-ans
2025-07-23 10:15:18,065 - WARNING - Article trop court ou vide : https://www.france24.com/fr/vid%C3%A9o/20250723-le-roi-du-heavy-metal-ozzy-osbourne-s-%C3%A9teint-%C3%A0-76-ans
2025-07-23 10:15:18,065 - INFO - Téléchargement de l'article : https://www.france24.com/fr/vid%C3%A9o/20250723-sable-eau-minerai-un-atlas-des-ressources-en-voie-de-disparition
2025-07-23 10:15:18,364 - INFO - Téléchargement de l'article : https://www.france24.com/fr/vid%C3%A9o/20250723-l-ukraine-adopte-une-loi-supprimant-l-ind%C3%A9pendance-des-instances-anticorruption
2025-07-23 10:15:18,556 - WARNING - Article trop court ou vide : https://www.france24.com/fr/vid%C3%A9o/20250723-l-ukraine-adopte-une-loi-supprimant-l-ind%C3%A9pendance-des-instances-anticorruption
2025-07-23 10:15:18,556 - INFO - Téléchargement de l'article : https://www.fra

...

Progression: 490/510 (96.1%) - Insérés: 362, Doublons: 0


2025-07-23 10:17:45,863 - INFO - Téléchargement de l'article : https://www.nytimes.com/2025/07/22/world/asia/bed-sharing-parents-children-sleep-asia.html
2025-07-23 10:17:46,296 - INFO - Téléchargement de l'article : https://www.nytimes.com/2025/07/21/briefing/bangladesh-plane-crash-gaza-israel-hamas-ukraine-russia.html
2025-07-23 10:17:47,249 - INFO - Téléchargement de l'article : https://www.nytimes.com/2025/07/21/climate/extreme-fire-weather-forests.html
2025-07-23 10:17:47,714 - INFO - Téléchargement de l'article : https://www.nytimes.com/2025/07/21/us/politics/china-exit-ban-us-government-employee.html
2025-07-23 10:17:48,055 - INFO - Téléchargement de l'article : https://www.nytimes.com/2025/07/21/climate/metals-company-seabed-mining-trump.html
2025-07-23 10:17:48,383 - INFO - Téléchargement de l'article : https://www.nytimes.com/2025/07/21/world/middleeast/syria-clashes-druse-bedouin-evacuations.html
2025-07-23 10:17:48,917 - INFO - Téléchargement de l'article : https://www.nyti

Progression: 500/510 (98.0%) - Insérés: 372, Doublons: 0


2025-07-23 10:17:51,301 - INFO - Téléchargement de l'article : https://www.nytimes.com/2025/07/21/world/asia/china-lead-poisoning-scandal.html
2025-07-23 10:17:51,693 - INFO - Téléchargement de l'article : https://www.nytimes.com/2025/07/21/world/middleeast/iran-nuclear-talks-europe.html
2025-07-23 10:17:52,415 - INFO - Téléchargement de l'article : https://www.nytimes.com/2025/07/21/world/europe/ukraine-weapons-industry.html
2025-07-23 10:17:52,870 - INFO - Téléchargement de l'article : https://www.nytimes.com/2025/07/21/climate/china-brazil-electric-vehicles.html
2025-07-23 10:17:53,329 - INFO - Téléchargement de l'article : https://www.nytimes.com/2025/07/21/world/europe/russia-ukraine-missile-drone-strikes.html
2025-07-23 10:17:53,713 - INFO - Téléchargement de l'article : https://www.nytimes.com/2025/07/21/world/asia/south-korea-floods-death-toll.html
2025-07-23 10:17:54,439 - INFO - Téléchargement de l'article : https://www.nytimes.com/2025/07/21/briefing/gaza-israel-japan-trump-

Progression: 510/510 (100.0%) - Insérés: 382, Doublons: 0

Résumé du traitement:
   • Articles traités: 382/510
   • Articles insérés: 382
   • Doublons détectés: 0
   • Erreurs: 0
   • Temps total: 167.17s
   • Taux de succès: 74.9%


## 3. Traitement et stockage

Extraction du texte complet et insertion en base de données avec gestion d'erreurs avancée.

In [31]:
# Création et enrichissement du DataFrame
if processed:
    df = pd.DataFrame(processed)

    # Nettoyage et enrichissement
    df["published"] = pd.to_datetime(df["published"], errors='coerce')
    df["title_length"] = df["title"].fillna("").str.len()
    df["text_length"] = df["text"].fillna("").str.len()
    df["summary_length"] = df["summary"].fillna("").str.len()

    # Extraction de features temporelles
    df["published_date"] = df["published"].dt.date
    df["published_hour"] = df["published"].dt.hour
    df["published_weekday"] = df["published"].dt.day_name()
    df["published_month"] = df["published"].dt.month

    # Qualité du texte
    df["words_count"] = df["text"].fillna("").str.split().str.len()
    df["reading_time"] = (df["words_count"] / 250).round(1)  # lecture estimée

    # Statistiques robustes
    print("Statistiques du dataset :")
    print(f"   • Nombre d'articles : {len(df)}")

    # Gestion du cas où toutes les dates sont NaT
    if df["published"].notna().sum() > 0:
        min_date = df["published"].min().strftime("%Y-%m-%d")
        max_date = df["published"].max().strftime("%Y-%m-%d")
        print(f"   • Période couverte : {min_date} à {max_date}")
    else:
        print("   • Période couverte : inconnue (aucune date valide)")

    print(f"   • Sources uniques : {df['source'].nunique()}")
    print(f"   • Articles avec date : {df['published'].notna().sum()}")

    print("\n Métriques de contenu :")
    print(f"   • Longueur moyenne titre : {df['title_length'].mean():.0f} caractères")
    print(f"   • Longueur moyenne texte : {df['text_length'].mean():.0f} caractères")
    print(f"   • Temps de lecture moyen : {df['reading_time'].mean():.1f} minutes")
    print(f"   • Articles > 1000 caractères : {(df['text_length'] > 1000).sum()}")

    # Affichage
    print("\nAperçu des articles :")
    display_cols = ['title', 'source', 'published', 'text_length', 'reading_time']
    print(df[display_cols].head(3).to_string())

else:
    print("Aucun article traité pour l'analyse.")
    df = pd.DataFrame()


Statistiques du dataset :
   • Nombre d'articles : 382
   • Période couverte : inconnue (aucune date valide)
   • Sources uniques : 8
   • Articles avec date : 0

 Métriques de contenu :
   • Longueur moyenne titre : 85 caractères
   • Longueur moyenne texte : 3506 caractères
   • Temps de lecture moyen : 2.2 minutes
   • Articles > 1000 caractères : 341

Aperçu des articles :
                                                                                                                                      title                              source published  text_length  reading_time
0                          L’expulsion silencieuse des Bédouins de Cisjordanie,  attaqués chaque jour par des colons extrémistes israéliens  https://www.lemonde.fr/rss/une.xml       NaT         1956           1.2
1                      L’entraîneur de ski Joël Chenal visé par une plainte pour agression sexuelle sur mineure et de nouvelles accusations  https://www.lemonde.fr/rss/une.xml       NaT         

In [33]:
print("Exemple de dates extraites :")
print(df["published"].dropna().head())
print("Dates valides :", df["published"].notna().sum(), "/", len(df))


Exemple de dates extraites :
Series([], Name: published, dtype: datetime64[ns])
Dates valides : 0 / 382


## 4. Analyses statistiques de base

Création du DataFrame et calcul des métriques de base pour l'analyse.

In [ ]:
# Analyses temporelles avancées
if not df.empty and df["published"].notna().sum() > 0:
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    fig.suptitle('Analyses Temporelles des Publications', fontsize=16, y=0.98)
    
    # 1. Distribution des longueurs de texte
    axes[0, 0].hist(df["text_length"], bins=30, alpha=0.7, color='skyblue', edgecolor='black')
    axes[0, 0].set_title("Distribution des longueurs de texte")
    axes[0, 0].set_xlabel("Nombre de caractères")
    axes[0, 0].set_ylabel("Nombre d'articles")
    axes[0, 0].axvline(df["text_length"].mean(), color='red', linestyle='--', 
                       label=f'Moyenne: {df["text_length"].mean():.0f}')
    axes[0, 0].legend()
    
    # 2. Publications par heure
    if df["published_hour"].notna().sum() > 0:
        hourly_counts = df["published_hour"].value_counts().sort_index()
        axes[0, 1].bar(hourly_counts.index, hourly_counts.values, color='lightgreen', alpha=0.7)
        axes[0, 1].set_title("Publications par heure de la journée")
        axes[0, 1].set_xlabel("Heure")
        axes[0, 1].set_ylabel("Nombre d'articles")
        axes[0, 1].set_xticks(range(0, 24, 2))
    
    # 3. Publications par jour de la semaine
    if df["published_weekday"].notna().sum() > 0:
        weekday_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
        weekday_counts = df["published_weekday"].value_counts().reindex(weekday_order, fill_value=0)
        axes[1, 0].bar(range(len(weekday_counts)), weekday_counts.values, 
                       color='orange', alpha=0.7)
        axes[1, 0].set_title("Publications par jour de la semaine")
        axes[1, 0].set_xlabel("Jour de la semaine")
        axes[1, 0].set_ylabel("Nombre d'articles")
        axes[1, 0].set_xticks(range(len(weekday_order)))
        axes[1, 0].set_xticklabels([day[:3] for day in weekday_order], rotation=45)
    
    # 4. Évolution temporelle
    if df["published_date"].notna().sum() > 0:
        daily_counts = df["published_date"].value_counts().sort_index()
        axes[1, 1].plot(daily_counts.index, daily_counts.values, 
                        marker='o', linewidth=2, markersize=4, color='purple')
        axes[1, 1].set_title("Évolution des publications dans le temps")
        axes[1, 1].set_xlabel("Date")
        axes[1, 1].set_ylabel("Nombre d'articles")
        axes[1, 1].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()
    
    # Statistiques temporelles
    print("Insights temporels:")
    if df["published_hour"].notna().sum() > 0:
        peak_hour = df["published_hour"].mode().iloc[0]
        print(f"   • Heure de publication la plus active: {peak_hour}h")
    
    if df["published_weekday"].notna().sum() > 0:
        peak_day = df["published_weekday"].mode().iloc[0]
        print(f"   • Jour le plus actif: {peak_day}")
    
    # Corrélations intéressantes
    correlations = df[['text_length', 'title_length', 'reading_time', 'published_hour']].corr()
    print(f"\n🔗 Corrélations notables:")
    print(f"   • Corrélation titre/texte: {correlations.loc['title_length', 'text_length']:.3f}")
    
else:
    print("Pas assez de données temporelles pour l'analyse")
    
    


Pas assez de données temporelles pour l'analyse


## 5. Analyses temporelles avancées

Analyse des patterns de publication dans le temps.

In [ ]:
# Analyse des mots-clés et thèmes
if not df.empty:
    
    # Préparation des données textuelles
    all_titles = ' '.join(df['title'].fillna('').str.lower())
    all_summaries = ' '.join(df['summary'].fillna('').str.lower())
    
    # Mots vides français + anglais (basic list)
    stop_words = {
        'le', 'de', 'et', 'à', 'un', 'il', 'être', 'et', 'en', 'avoir', 'que', 'pour',
        'dans', 'ce', 'son', 'une', 'sur', 'avec', 'ne', 'se', 'pas', 'tout', 'plus',
        'par', 'grand', 'comme', 'mais', 'faire', 'son', 'que', 'du', 'sa', 'les',
        'des', 'la', 'au', 'aux', 'nous', 'vous', 'ils', 'elle', 'sont', 'ont', 'est',
        'the', 'and', 'or', 'but', 'in', 'on', 'at', 'to', 'for', 'of', 'with',
        'by', 'an', 'a', 'is', 'are', 'was', 'were', 'be', 'been', 'have', 'has', 'had'
    }
    
    # Extraction des mots-clés des titres
    title_words = re.findall(r'\\b[a-zA-Zàâäéèêëïîôöùûüç]{3,}\\b', all_titles)
    title_words = [word for word in title_words if word not in stop_words]
    title_word_freq = Counter(title_words).most_common(20)
    
    # Extraction des mots-clés des résumés
    summary_words = re.findall(r'\\b[a-zA-Zàâäéèêëïîôöùûüç]{3,}\\b', all_summaries)
    summary_words = [word for word in summary_words if word not in stop_words]
    summary_word_freq = Counter(summary_words).most_common(20)
    
    # Visualisation des mots-clés
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    fig.suptitle('Analyse des Mots-Clés', fontsize=16)
    
    # Mots-clés des titres
    if title_word_freq:
        words, counts = zip(*title_word_freq[:15])
        axes[0].barh(range(len(words)), counts, color='lightblue')
        axes[0].set_yticks(range(len(words)))
        axes[0].set_yticklabels(words)
        axes[0].set_xlabel('Fréquence')
        axes[0].set_title('Mots les plus fréquents dans les titres')
        axes[0].invert_yaxis()
    
    # Mots-clés des résumés
    if summary_word_freq:
        words, counts = zip(*summary_word_freq[:15])
        axes[1].barh(range(len(words)), counts, color='lightcoral')
        axes[1].set_yticks(range(len(words)))
        axes[1].set_yticklabels(words)
        axes[1].set_xlabel('Fréquence')
        axes[1].set_title('Mots les plus fréquents dans les résumés')
        axes[1].invert_yaxis()
    
    plt.tight_layout()
    plt.show()
    
    # Analyse des longueurs et diversité
    print("Insights textuels:")
    print(f"   • Mots uniques dans les titres: {len(set(title_words))}")
    print(f"   • Mots uniques dans les résumés: {len(set(summary_words))}")
    print(f"   • Diversité lexicale titres: {len(set(title_words))/len(title_words):.3f}")
    
    # Top mots-clés par catégorie
    print(f"\\n  Top 5 mots-clés titres:")
    for word, count in title_word_freq[:5]:
        print(f"   • {word}: {count} occurrences")
    
    print(f"\\n  Top 5 mots-clés résumés:")
    for word, count in summary_word_freq[:5]:
        print(f"   • {word}: {count} occurrences")
    
    # Analyse de la diversité des sources
    source_diversity = df['source'].value_counts()
    print(f"\\n Diversité des sources:")
    print(f"   • Source principale: {source_diversity.index[0]} ({source_diversity.iloc[0]} articles)")
    print(f"   • Répartition: {(source_diversity / len(df) * 100).round(1).to_dict()}")
    
else:
    print(" Pas de données pour l'analyse textuelle")

## 6. Analyse des mots-clés et contenus

Analyse textuelle pour identifier les thèmes et mots-clés dominants.

In [ ]:
# Visualisations interactives avec Plotly
if not df.empty:
    
    # 1. Graphique scatter interactif: longueur texte vs titre
    fig1 = px.scatter(
        df, 
        x='text_length', 
        y='title_length',
        color='source',
        size='reading_time',
        hover_data=['title', 'published', 'words_count'],
        title=' Relation Longueur Texte vs Longueur Titre (par source)',
        labels={
            'text_length': 'Longueur du texte (caractères)',
            'title_length': 'Longueur du titre (caractères)',
            'source': 'Source RSS'
        }
    )
    fig1.update_layout(height=500)
    fig1.show()
    
    # 2. Graphique temporel interactif
    if df["published"].notna().sum() > 10:
        # Agrégation par jour
        daily_stats = df.groupby(df['published'].dt.date).agg({
            'title': 'count',
            'text_length': 'mean',
            'reading_time': 'mean',
            'source': lambda x: x.nunique()
        }).rename(columns={
            'title': 'articles_count',
            'text_length': 'avg_length',
            'reading_time': 'avg_reading_time',
            'source': 'unique_sources'
        })
        
        fig2 = make_subplots(
            rows=2, cols=2,
            subplot_titles=('Articles par jour', 'Longueur moyenne', 
                          'Temps de lecture moyen', 'Sources uniques par jour'),
            vertical_spacing=0.08
        )
        
        # Articles par jour
        fig2.add_trace(
            go.Scatter(x=daily_stats.index, y=daily_stats['articles_count'],
                      mode='lines+markers', name='Articles/jour', line=dict(color='blue')),
            row=1, col=1
        )
        
        # Longueur moyenne
        fig2.add_trace(
            go.Scatter(x=daily_stats.index, y=daily_stats['avg_length'],
                      mode='lines+markers', name='Longueur moy.', line=dict(color='green')),
            row=1, col=2
        )
        
        # Temps de lecture moyen
        fig2.add_trace(
            go.Scatter(x=daily_stats.index, y=daily_stats['avg_reading_time'],
                      mode='lines+markers', name='Temps lecture', line=dict(color='red')),
            row=2, col=1
        )
        
        # Sources uniques
        fig2.add_trace(
            go.Bar(x=daily_stats.index, y=daily_stats['unique_sources'],
                   name='Sources uniques', marker_color='orange'),
            row=2, col=2
        )
        
        fig2.update_layout(
            height=600,
            title_text=" Évolution Temporelle des Métriques",
            showlegend=False
        )
        fig2.show()
    
    # 3. Distribution des sources (graphique en secteurs interactif)
    source_counts = df['source'].value_counts()
    fig3 = px.pie(
        values=source_counts.values,
        names=source_counts.index,
        title=' Distribution des Articles par Source RSS',
        hover_data=[source_counts.values],
        labels={'values': f"Nombre d'articles"}
    )
    fig3.update_traces(textposition='inside', textinfo='percent+label')
    fig3.update_layout(height=500)
    fig3.show()
    
    # 4. Heatmap des publications par heure et jour
    if df["published_hour"].notna().sum() > 0 and df["published_weekday"].notna().sum() > 0:
        heatmap_data = df.groupby(['published_weekday', 'published_hour']).size().reset_index(name='count')
        heatmap_pivot = heatmap_data.pivot(index='published_weekday', 
                                         columns='published_hour', 
                                         values='count').fillna(0)
        
        # Réorganiser les jours de la semaine
        weekday_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 
                        'Friday', 'Saturday', 'Sunday']
        heatmap_pivot = heatmap_pivot.reindex(weekday_order, fill_value=0)
        
        fig4 = px.imshow(
            heatmap_pivot,
            labels=dict(x="Heure", y="Jour", color="Articles"),
            title="Heatmap: Publications par Jour et Heure",
            color_continuous_scale="Viridis"
        )
        fig4.update_layout(height=400)
        fig4.show()
    
    print("Toutes les visualisations interactives ont été générées!")
    
else:
    print("Pas de données pour les visualisations interactives")

## 8. Métriques de qualité

Évaluation de la performance du système de collecte et de la qualité des données.

In [ ]:
# Rapport complet des métriques de qualité
print("RAPPORT DE QUALITÉ - INSIGHTDETECTOR")
print("=" * 50)

# Métriques de collecte
if 'metrics' in locals():
    print("\\n MÉTRIQUES DE COLLECTE:")
    success_rate = (metrics['total_inserted'] / max(metrics['total_fetched'], 1)) * 100
    print(f"   • Taux de succès global: {success_rate:.1f}%")
    print(f"   • Articles récupérés: {metrics['total_fetched']}")
    print(f"   • Articles traités: {metrics['total_processed']}")  
    print(f"   • Articles insérés: {metrics['total_inserted']}")
    print(f"   • Doublons détectés: {metrics['total_duplicates']}")
    print(f"   • Erreurs rencontrées: {len(metrics['processing_errors'])}")
    
    # Performance par source
    if metrics['sources_success']:
        print(f"\\n PERFORMANCE PAR SOURCE:")
        total_by_source = metrics['sources_success']
        for source, count in sorted(total_by_source.items(), key=lambda x: x[1], reverse=True):
            percentage = (count / metrics['total_fetched']) * 100
            print(f"   • {source[:30]:<30} : {count:>3} articles ({percentage:>5.1f}%)")

# Métriques de qualité des données
if not df.empty:
    print(f"\\n QUALITÉ DES DONNÉES:")
    
    # Complétude des données
    completeness = {
        'Titre': df['title'].notna().sum() / len(df),
        'URL': df['url'].notna().sum() / len(df),
        'Résumé': df['summary'].notna().sum() / len(df),
        'Texte': df['text'].notna().sum() / len(df),
        'Date publication': df['published'].notna().sum() / len(df),
        'Source': df['source'].notna().sum() / len(df)
    }
    
    for field, completeness_rate in completeness.items():
        status = "✅" if completeness_rate > 0.9 else "⚠️ " if completeness_rate > 0.7 else "❌"
        print(f"   • {field:<15}: {completeness_rate:>6.1%} {status}")
    
    # Qualité du contenu
    print(f"\\n QUALITÉ DU CONTENU:")
    short_articles = (df['text_length'] < 500).sum()
    medium_articles = ((df['text_length'] >= 500) & (df['text_length'] < 2000)).sum()
    long_articles = (df['text_length'] >= 2000).sum()
    
    print(f"   • Articles courts (<500 char): {short_articles} ({short_articles/len(df)*100:.1f}%)")
    print(f"   • Articles moyens (500-2000): {medium_articles} ({medium_articles/len(df)*100:.1f}%)")
    print(f"   • Articles longs (>2000 char): {long_articles} ({long_articles/len(df)*100:.1f}%)")
    
    # Diversité temporelle
    if df['published'].notna().sum() > 0:
        date_range = (df['published'].max() - df['published'].min()).days
        unique_days = df['published'].dt.date.nunique()
        print(f"\\n COUVERTURE TEMPORELLE:")
        print(f"   • Période analysée: {date_range} jours")
        print(f"   • Jours avec publications: {unique_days}")
        print(f"   • Couverture temporelle: {unique_days/max(date_range, 1)*100:.1f}%")
    
    # Analyse des anomalies
    print(f"\\n DÉTECTION D'ANOMALIES:")
    
    # Articles avec titre très long/court
    very_short_titles = (df['title_length'] < 20).sum()
    very_long_titles = (df['title_length'] > 150).sum()
    print(f"   • Titres très courts (<20 char): {very_short_titles}")
    print(f"   • Titres très longs (>150 char): {very_long_titles}")
    
    # URLs potentiellement problématiques
    duplicate_urls = df['url'].duplicated().sum()
    print(f"   • URLs dupliquées: {duplicate_urls}")
    
    # Contenu potentiellement vide ou invalide
    empty_content = ((df['text'].str.len() < 100) | (df['text'].isna())).sum()
    print(f"   • Contenu vide/invalide: {empty_content}")

# Score de qualité global
if not df.empty and 'metrics' in locals():
    print(f"\\n SCORE DE QUALITÉ GLOBAL:")
    
    # Calcul du score (sur 100)
    collection_score = min(success_rate, 100) * 0.3  # 30% pour la collecte
    completeness_score = (sum(completeness.values()) / len(completeness)) * 100 * 0.4  # 40% pour la complétude
    content_score = ((medium_articles + long_articles) / len(df)) * 100 * 0.3  # 30% pour la qualité du contenu
    
    total_score = collection_score + completeness_score + content_score
    
    print(f"   • Score de collecte: {collection_score:.1f}/30")
    print(f"   • Score de complétude: {completeness_score:.1f}/40") 
    print(f"   • Score de contenu: {content_score:.1f}/30")
    print(f"   • SCORE TOTAL: {total_score:.1f}/100")
    
    # Recommandations
    print(f"\\n RECOMMANDATIONS:")
    if total_score >= 80:
        print("    Excellente qualité des données!")
    elif total_score >= 60:
        print("     Qualité correcte, quelques améliorations possibles:")
        if success_rate < 80:
            print("      - Améliorer le taux de succès de collecte")
        if completeness_score < 35:
            print("      - Améliorer la complétude des champs")
        if content_score < 25:
            print("      - Filtrer davantage le contenu de faible qualité")
    else:
        print("    Qualité insuffisante, actions requises:")
        print("      - Revoir la configuration des sources RSS")
        print("      - Améliorer les filtres de qualité")
        print("      - Corriger les erreurs de traitement")

print("\\n" + "=" * 50)

## 9. Conclusions et recommandations

Synthèse des insights découverts et recommandations pour améliorer le système.

In [ ]:
# Nettoyage des ressources et conclusion
try:
    if session:
        session.close()
        logger.info("🧹 Session de base de données fermée")
    
    # Résumé final pour l'utilisateur
    if not df.empty and 'metrics' in locals():
        print("\\n ANALYSE TERMINÉE AVEC SUCCÈS!")
        print(f" {len(df)} articles analysés de {df['source'].nunique()} sources différentes")
        print(f" Période couverte: {(df['published'].max() - df['published'].min()).days} jours")
        print(f" Taux de réussite: {(metrics['total_inserted']/metrics['total_fetched']*100):.1f}%")
        
        # Insights clés découverts
        print(f"\\n INSIGHTS CLÉS:")
        if df['published_hour'].notna().sum() > 0:
            peak_hour = df['published_hour'].mode().iloc[0]
            print(f"   • Heure de pointe: {peak_hour}h")
        
        top_source = df['source'].mode().iloc[0]
        top_source_count = df['source'].value_counts().iloc[0]
        print(f"   • Source la plus productive: {top_source} ({top_source_count} articles)")
        
        avg_length = df['text_length'].mean()
        print(f"   • Longueur moyenne des articles: {avg_length:.0f} caractères")
        
        avg_reading = df['reading_time'].mean()
        print(f"   • Temps de lecture moyen: {avg_reading:.1f} minutes")
    
    print("\\n Toutes les analyses sont complètes. Le notebook peut être utilisé pour:")
    print("   • Surveiller la qualité de la collecte RSS")
    print("   • Identifier les tendances temporelles de publication") 
    print("   • Analyser les performances par source")
    print("   • Détecter les anomalies dans les données")
    print("   • Optimiser la configuration des flux RSS")
    
except Exception as e:
    logger.error(f"Erreur lors du nettoyage: {e}")
    
finally:
    print("\\n Analyse RSS terminée.")

## 7. Visualisations interactives

Graphiques interactifs pour une exploration approfondie des données.

In [ ]:
# Visualisations interactives avec Plotly (avec fallback)
if not df.empty:
    
    if plotly_available:
        print("🎨 Génération des visualisations interactives Plotly...")
        
        # 1. Graphique scatter interactif: longueur texte vs titre
        fig1 = px.scatter(
            df, 
            x='text_length', 
            y='title_length',
            color='source',
            size='reading_time',
            hover_data=['title', 'published', 'words_count'],
            title='📊 Relation Longueur Texte vs Longueur Titre (par source)',
            labels={
                'text_length': 'Longueur du texte (caractères)',
                'title_length': 'Longueur du titre (caractères)',
                'source': 'Source RSS'
            }
        )
        fig1.update_layout(height=500)
        fig1.show()
        
        # 2. Graphique temporel interactif
        if df["published"].notna().sum() > 10:
            # Agrégation par jour
            daily_stats = df.groupby(df['published'].dt.date).agg({
                'title': 'count',
                'text_length': 'mean',
                'reading_time': 'mean',
                'source': lambda x: x.nunique()
            }).rename(columns={
                'title': 'articles_count',
                'text_length': 'avg_length',
                'reading_time': 'avg_reading_time',
                'source': 'unique_sources'
            })
            
            fig2 = make_subplots(
                rows=2, cols=2,
                subplot_titles=('Articles par jour', 'Longueur moyenne', 
                              'Temps de lecture moyen', 'Sources uniques par jour'),
                vertical_spacing=0.08
            )
            
            # Articles par jour
            fig2.add_trace(
                go.Scatter(x=daily_stats.index, y=daily_stats['articles_count'],
                          mode='lines+markers', name='Articles/jour', line=dict(color='blue')),
                row=1, col=1
            )
            
            # Longueur moyenne
            fig2.add_trace(
                go.Scatter(x=daily_stats.index, y=daily_stats['avg_length'],
                          mode='lines+markers', name='Longueur moy.', line=dict(color='green')),
                row=1, col=2
            )
            
            # Temps de lecture moyen
            fig2.add_trace(
                go.Scatter(x=daily_stats.index, y=daily_stats['avg_reading_time'],
                          mode='lines+markers', name='Temps lecture', line=dict(color='red')),
                row=2, col=1
            )
            
            # Sources uniques
            fig2.add_trace(
                go.Bar(x=daily_stats.index, y=daily_stats['unique_sources'],
                       name='Sources uniques', marker_color='orange'),
                row=2, col=2
            )
            
            fig2.update_layout(
                height=600,
                title_text="📈 Évolution Temporelle des Métriques",
                showlegend=False
            )
            fig2.show()
        
        # 3. Distribution des sources (graphique en secteurs interactif)
        source_counts = df['source'].value_counts()
        fig3 = px.pie(
            values=source_counts.values,
            names=source_counts.index,
            title='🥧 Distribution des Articles par Source RSS',
            hover_data=[source_counts.values],
            labels={'values': 'Nombre d\'articles'}
        )
        fig3.update_traces(textposition='inside', textinfo='percent+label')
        fig3.update_layout(height=500)
        fig3.show()
        
        # 4. Heatmap des publications par heure et jour
        if df["published_hour"].notna().sum() > 0 and df["published_weekday"].notna().sum() > 0:
            heatmap_data = df.groupby(['published_weekday', 'published_hour']).size().reset_index(name='count')
            heatmap_pivot = heatmap_data.pivot(index='published_weekday', 
                                             columns='published_hour', 
                                             values='count').fillna(0)
            
            # Réorganiser les jours de la semaine
            weekday_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 
                            'Friday', 'Saturday', 'Sunday']
            heatmap_pivot = heatmap_pivot.reindex(weekday_order, fill_value=0)
            
            fig4 = px.imshow(
                heatmap_pivot,
                labels=dict(x="Heure", y="Jour", color="Articles"),
                title="🔥 Heatmap: Publications par Jour et Heure",
                color_continuous_scale="Viridis"
            )
            fig4.update_layout(height=400)
            fig4.show()
        
        print("✅ Visualisations Plotly générées avec succès!")
        
    else:
        # Fallback avec matplotlib si Plotly n'est pas disponible
        print("📊 Plotly non disponible - utilisation de matplotlib...")
        
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        fig.suptitle('📊 Analyses Visuelles des Données RSS', fontsize=16)
        
        # 1. Scatter plot longueur texte vs titre
        scatter = axes[0, 0].scatter(df['text_length'], df['title_length'], 
                                   c=df['reading_time'], alpha=0.6, cmap='viridis')
        axes[0, 0].set_xlabel('Longueur texte (caractères)')
        axes[0, 0].set_ylabel('Longueur titre (caractères)')
        axes[0, 0].set_title('Relation Texte vs Titre')
        plt.colorbar(scatter, ax=axes[0, 0], label='Temps lecture (min)')
        
        # 2. Distribution des longueurs
        axes[0, 1].hist(df['text_length'], bins=30, alpha=0.7, color='skyblue', label='Texte')
        axes[0, 1].hist(df['title_length'], bins=30, alpha=0.7, color='orange', label='Titre')
        axes[0, 1].set_xlabel('Longueur (caractères)')
        axes[0, 1].set_ylabel('Fréquence')
        axes[0, 1].set_title('Distribution des longueurs')
        axes[0, 1].legend()
        
        # 3. Articles par source
        source_counts = df['source'].value_counts()
        axes[1, 0].barh(range(len(source_counts)), source_counts.values, color='lightgreen')
        axes[1, 0].set_yticks(range(len(source_counts)))
        axes[1, 0].set_yticklabels([s[:20] + '...' if len(s) > 20 else s for s in source_counts.index])
        axes[1, 0].set_xlabel('Nombre d\'articles')
        axes[1, 0].set_title('Articles par source')
        
        # 4. Évolution temporelle si données disponibles
        if df['published'].notna().sum() > 5:
            daily_counts = df.groupby(df['published'].dt.date).size()
            axes[1, 1].plot(daily_counts.index, daily_counts.values, marker='o')
            axes[1, 1].set_xlabel('Date')
            axes[1, 1].set_ylabel('Nombre d\'articles')
            axes[1, 1].set_title('Évolution temporelle')
            axes[1, 1].tick_params(axis='x', rotation=45)
        else:
            axes[1, 1].text(0.5, 0.5, 'Pas assez de\ndonnées temporelles', 
                           ha='center', va='center', transform=axes[1, 1].transAxes)
            axes[1, 1].set_title('Évolution temporelle')
        
        plt.tight_layout()
        plt.show()
        
        print("✅ Visualisations matplotlib générées!")
    
    print("🎯 Analyses visuelles terminées avec succès!")
    
else:
    print("❌ Pas de données pour les visualisations")